In [1]:
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification

In [21]:
data = pd.read_csv('../Anomaly-Detection/amazon_reviews.txt', delimiter="\t")
features = ['RATING', 'REVIEW_TEXT', 'VERIFIED_PURCHASE', 'LABEL']


#data_true = data[  data['LABEL'] == '__label1__' ]

data_shortened = data[features]

data_shortened['VERIFIED_PURCHASE'] = data_shortened['VERIFIED_PURCHASE'].replace('N', 0)
data_shortened['VERIFIED_PURCHASE'] = data_shortened['VERIFIED_PURCHASE'].replace('Y', 1)

data_shortened['LABEL'] = data_shortened['LABEL'].replace('__label1__', 1)
data_shortened['LABEL'] = data_shortened['LABEL'].replace('__label2__', 0)

#data_pos = data_shortened[  data_shortened['RATING'] == 5 ]
#data_neg = data_shortened[data_shortened['RATING'] == 1]

#print(data_pos.head())

labels = data_shortened.pop('LABEL')

#print(data_pos.head())

#print(data_pos.dtypes)

<ipython-input-21-5d0a6fe65350>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_shortened['VERIFIED_PURCHASE'] = data_shortened['VERIFIED_PURCHASE'].replace('N', 0)
<ipython-input-21-5d0a6fe65350>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_shortened['VERIFIED_PURCHASE'] = data_shortened['VERIFIED_PURCHASE'].replace('Y', 1)
<ipython-input-21-5d0a6fe65350>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [25]:

import numpy as np

reviews_array = data_shortened["REVIEW_TEXT"].to_numpy(dtype=object)
labels_array = labels.to_numpy(dtype=np.int32)

#Shuffling the dataset
data_length = len(reviews_array)
idx = np.random.permutation(data_length)
shuffled_reviews_array = reviews_array[idx]
shuffled_labels_array = labels_array[idx]

# Train-test split
train_ratio = 0.8
train_data_len = int( train_ratio * data_length )

train_reviews_array = shuffled_reviews_array[:train_data_len]
train_labels_array = shuffled_labels_array[:train_data_len]

test_reviews_array = shuffled_reviews_array[train_data_len:]
test_labels_array = shuffled_reviews_array[train_data_len:]


tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
#tokenized_df = tokenizer(data_shortened.head()["REVIEW_TEXT"].tolist(), padding="max_length", truncation=True)

train_reviews = train_reviews_array.tolist()

tokenized_df = tokenizer(train_reviews[0:5], padding="max_length", truncation=True)
#print( np.sum(train_labels_array == 0))
print(train_labels_array[0:5])

print(type(tokenized_df))
print(tokenized_df.keys())

8411
[0 1 0 0 0]
<class 'transformers.tokenization_utils_base.BatchEncoding'>
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [28]:

train_features = {x: tf.convert_to_tensor(tokenized_df[x]) for x in tokenizer.model_input_names}
#print(train_features)

#The dataset now is a tuple of dictionary containint inpud_ids, token_type_ids. attention masks        and  a tf tensor containing the value for the label

#train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, labels.head()))
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels_array[0:5]))

#print(train_tf_dataset)
train_tf_dataset = train_tf_dataset.shuffle(5).batch(5)

print(train_tf_dataset)



<BatchDataset shapes: ({input_ids: (None, 512), token_type_ids: (None, 512), attention_mask: (None, 512)}, (None,)), types: ({input_ids: tf.int32, token_type_ids: tf.int32, attention_mask: tf.int32}, tf.int32)>


In [32]:
model = TFAutoModelForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2)


model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(train_tf_dataset, validation_data=train_tf_dataset, epochs=10, verbose=1) #3 epochs

model.save_pretrained("transformer_model")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
1/1 [==============================] - 9s 9s/step - loss: 0.5692 - sparse_categorical_accuracy: 0.8000 - val_loss: 0.5164 - val_sparse_categorical_accuracy: 0.8000
Epoch 2/10
1/1 [==============================] - 4s 4s/step - loss: 0.5651 - sparse_categorical_accuracy: 0.8000 - val_loss: 0.4279 - val_sparse_categorical_accuracy: 0.8000
Epoch 3/10
1/1 [==============================] - 4s 4s/step - loss: 0.4575 - sparse_categorical_accuracy: 0.8000 - val_loss: 0.3126 - val_sparse_categorical_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 3s 3s/step - loss: 0.3183 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.2402 - val_sparse_categorical_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 4s 4s/step - loss: 0.2462 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.2058 - val_sparse_categorical_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 4s 4s/step - loss: 0.2281 - sparse_categorical_accuracy: 1.0000 - val_loss:

In [34]:

#print(data_pos.head()["REVIEW_TEXT"].tolist()[0])

print(train_labels_array[0:5])

for i in range(5):
    #y = tokenizer(data_shortened.head()["REVIEW_TEXT"].tolist()[i], padding="max_length", truncation=True)
    y = tokenizer(train_reviews[i], padding="max_length", truncation=True)

    y['input_ids'] = tf.convert_to_tensor( [y['input_ids']] )
    y['token_type_ids'] = tf.convert_to_tensor( [y['token_type_ids']] )
    y['attention_mask'] = tf.convert_to_tensor( [y['attention_mask']] )

    label = model(y)
    probs = tf.nn.softmax(label.logits)
    print(probs)



[0 1 0 0 0]


InvalidArgumentError: Input to reshape is a tensor with 768 values, but the requested shape has 512 [Op:Reshape]